In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scipy.stats

In [3]:
1-scipy.stats.norm.cdf(-1.5)*2

0.8663855974622838

In [4]:
import sys
sys.path.append("../")
sys.path.append("../misc")
import pandas as pd
from misc.MoviaBusDataset import MoviaBusDataset
from misc.data_loader import load_network, adjacency_matrix
import pickle
import os
import yaml

In [5]:
#df = pd.read_hdf("../DCRNN/data/df_highway_2012_4mon_sample.h5")
base_dir = '../DCRNN/movia'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
if not os.path.exists(os.path.join(base_dir,'out')):
    os.makedirs(os.path.join(base_dir,'out'))
if not os.path.exists(os.path.join(base_dir,'data')):
    os.makedirs(os.path.join(base_dir,'data'))
if not os.path.exists(os.path.join(base_dir,'model')):
    os.makedirs(os.path.join(base_dir,'model'))

In [6]:
movia_train = MoviaBusDataset('../data/train/', interpolation=True, agg_time=10)
movia_val =  MoviaBusDataset('../data/validation/', interpolation=True, agg_time=10)
movia_test =  MoviaBusDataset('../data/test/', interpolation=True, agg_time=10)

movia_train.remove_trend()
movia_val.remove_trend(movia_train._historical_average)
movia_test.remove_trend(movia_train._historical_average)

#movia_dataset.remove_trend()
#data_file = os.path.join(base_dir,'movia.h5')
#all_data.to_hdf(data_file,key='df',format='table')


In [7]:
#road_network = load_network()
adj_mx = movia_train.adjacency_matrix(add_identity=True, weigh_bad=True)
#adj_mx = adj_mx.T
adj_mx = adj_mx.astype('float32')
#adj_mx = adj_mx * 100
with open(os.path.join(base_dir,'adj_mx.pkl'), 'wb') as f:
    pickle.dump([-1, -1, adj_mx], f, protocol=2)

In [8]:
#np.linalg.matrix_rank(adj_mx + np.diag([1 for _ in adj_mx]))

In [9]:
#np.linalg.matrix_rank(adj_mx.T+ np.diag([1 for _ in adj_mx]))

In [10]:
#np.linalg.matrix_rank(adj_mx)

In [11]:
import sys
sys.path.append('../DCRNN')
sys.path.append('../misc')
from lib.utils import calculate_random_walk_matrix
from MoviaBusDataset import MoviaBusDataset

In [12]:
import matplotlib.pyplot as plt
plt.figure
plt.imshow(adj_mx)

In [13]:
##TAKEN FROM generate_training_data.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import argparse
import numpy as np
import os
import pandas as pd


def generate_graph_seq2seq_io_data(
        df, x_offsets, y_offsets, add_time_in_day=True, add_day_in_week=False, scaler=None
):
    """
    Generate samples from
    :param df:
    :param x_offsets:
    :param y_offsets:
    :param add_time_in_day:
    :param add_day_in_week:
    :param scaler:
    :return:
    # x: (epoch_size, input_length, num_nodes, input_dim)
    # y: (epoch_size, output_length, num_nodes, output_dim)
    """

    num_samples, num_nodes = df.shape
    data = np.expand_dims(df.values, axis=-1)
    data_list = [data]
    if add_time_in_day:
        time_ind = (df.index.values - df.index.values.astype("datetime64[D]")) / np.timedelta64(1, "D")
        time_in_day = np.tile(time_ind, [1, num_nodes, 1]).transpose((2, 1, 0))
        data_list.append(time_in_day)
    if add_day_in_week:
        day_in_week = np.zeros(shape=(num_samples, num_nodes, 7))
        day_in_week[np.arange(num_samples), :, df.index.dayofweek] = 1
        data_list.append(day_in_week)

    data = np.concatenate(data_list, axis=-1)
    # epoch_len = num_samples + min(x_offsets) - max(y_offsets)
    x, y = [], []
    # t is the index of the last observation.
    min_t = abs(min(x_offsets))
    max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
    for t in range(min_t, max_t):
        x_t = data[t + x_offsets, ...]
        y_t = data[t + y_offsets, ...]
        x.append(x_t)
        y.append(y_t)
    x = np.stack(x, axis=0)
    y = np.stack(y, axis=0)
    return x, y


#def generate_train_val_test(args):
#df = pd.read_hdf(args.traffic_df_filename)
# 0 is the latest observed sample.
x_offsets = np.sort(
    # np.concatenate(([-week_size + 1, -day_size + 1], np.arange(-11, 1, 1)))
    #ADD +6 to go from 5*12 = 60mins of predtions to 10*6=60mins of predictions
    np.concatenate((np.arange(-11+6, 1, 1),))
)
#ADD -6 to go from 5*12 = 60mins of predtions to 10*6=60mins of predictions
y_offsets = np.sort(np.arange(1, 13-6, 1))
# x: (num_samples, input_length, num_nodes, input_dim)
# y: (num_samples, output_length, num_nodes, output_dim)
movia_train_df = pd.concat(movia_train.dataframes)
movia_train_df[movia_train_df.columns] = movia_train_df[movia_train_df.columns].astype('float32')

x_train, y_train = generate_graph_seq2seq_io_data(
    movia_train_df,
    x_offsets=x_offsets,
    y_offsets=y_offsets,
    add_time_in_day=True,
    add_day_in_week=False,
)
movia_val_df = pd.concat(movia_val.dataframes)
movia_val_df[movia_val_df.columns] = movia_val_df[movia_val_df.columns].astype('float32')

x_val, y_val = generate_graph_seq2seq_io_data(
    movia_val_df,
    x_offsets=x_offsets,
    y_offsets=y_offsets,
    add_time_in_day=True,
    add_day_in_week=False,
)

movia_test_df = pd.concat(movia_test.dataframes)
movia_test_df[movia_test_df.columns] = movia_test_df[movia_test_df.columns].astype('float32')

x_test, y_test = generate_graph_seq2seq_io_data(
    movia_test_df,
    x_offsets=x_offsets,
    y_offsets=y_offsets,
    add_time_in_day=True,
    add_day_in_week=False,
)

for cat in ["train", "val", "test"]:
    _x, _y = locals()["x_" + cat], locals()["y_" + cat]
    print(cat, "x: ", _x.shape, "y:", _y.shape)
    np.savez_compressed(
        os.path.join(os.path.join(base_dir,'data'), "%s.npz" % cat),
        x=_x,
        y=_y,
        x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
        y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
    )


train x:  (1519, 6, 192, 2) y: (1519, 6, 192, 2)
val x:  (193, 6, 192, 2) y: (193, 6, 192, 2)
test x:  (295, 6, 192, 2) y: (295, 6, 192, 2)


In [14]:
config = {
    'base_dir': 'data/model',
    'log_level': 'INFO',
    'data': {
      'batch_size': 32,
      'dataset_dir': 'movia/data',
      'test_batch_size': 32,
      'val_batch_size': 32,
      'graph_pkl_filename': 'movia/adj_mx.pkl'
    },
    'model': {
      'cl_decay_steps': 2000,
      'filter_type': 'random_walk',
      'horizon': 6,
      'input_dim': 2,
      'l1_decay': 0,
      'max_diffusion_step': 2,
      'num_nodes': len(adj_mx),
      'num_rnn_layers': 2,
      'output_dim': 1,
      'rnn_units': 64,
      'seq_len': 6,
      'use_curriculum_learning': True
    },
    'train': {
      'base_lr': 0.01,
      'dropout': 0,
      'epoch': 0,
      'epochs': 100,
      'epsilon': 1.0e-3,
      'global_step': 0,
      'lr_decay_ratio': 0.1,
      'max_grad_norm': 5,
      'max_to_keep': 100,
      'min_learning_rate': 2.0e-06,
      'optimizer': 'adam',
      'patience': 50,
      'steps': [20, 30, 40, 50],
      'test_every_n_epochs': 10
    }
}

with open(os.path.join(base_dir,'config.yml'), 'w') as outfile:
    yaml.dump(config, outfile, default_flow_style=False)

In [15]:
#IF running into ValueError: Variable rnn/basic_rnn_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at
#Try restarting kernel
os.chdir(os.path.join(base_dir,'..'))
#%run run_demo.py --config_filename movia/config.yml --output_filename movia/out/out.npz
import tensorflow as tf
import numpy as np

from lib.utils import load_graph_data
from model.dcrnn_supervisor import DCRNNSupervisor
use_cpu_only = False
graph_pkl_filename = 'movia/adj_mx.pkl'
tf_config = tf.ConfigProto()

if use_cpu_only:
    tf_config = tf.ConfigProto(device_count={'GPU': 0})
tf_config.gpu_options.allow_growth = True
_, _, adj_mx = load_graph_data(graph_pkl_filename)
supervisor = None
with tf.Session(config=tf_config) as sess:
    supervisor = DCRNNSupervisor(adj_mx=adj_mx, **config)
    supervisor.train(sess)
    #supervisor.load(sess, config['train']['model_filename'])
    outputs = supervisor.evaluate(sess)
    np.savez_compressed(os.path.join(base_dir,'out.npz'), **outputs)
    print('Predictions saved as {}.'.format(os.path.join(base_dir,'out.npz')))

2018-12-21 14:24:24,915 - INFO - Log directory: data/model/dcrnn_R_2_h_6_64-64_lr_0.01_bs_32_1221142424/
2018-12-21 14:24:24,918 - INFO - {'base_dir': 'data/model', 'log_level': 'INFO', 'data': {'batch_size': 32, 'dataset_dir': 'movia/data', 'test_batch_size': 32, 'val_batch_size': 32, 'graph_pkl_filename': 'movia/adj_mx.pkl'}, 'model': {'cl_decay_steps': 2000, 'filter_type': 'random_walk', 'horizon': 6, 'input_dim': 2, 'l1_decay': 0, 'max_diffusion_step': 2, 'num_nodes': 192, 'num_rnn_layers': 2, 'output_dim': 1, 'rnn_units': 64, 'seq_len': 6, 'use_curriculum_learning': True}, 'train': {'base_lr': 0.01, 'dropout': 0, 'epoch': 0, 'epochs': 100, 'epsilon': 0.001, 'global_step': 0, 'lr_decay_ratio': 0.1, 'max_grad_norm': 5, 'max_to_keep': 100, 'min_learning_rate': 2e-06, 'optimizer': 'adam', 'patience': 50, 'steps': [20, 30, 40, 50], 'test_every_n_epochs': 10}}
2018-12-21 14:24:25,223 - INFO - ('x_train', (1519, 6, 192, 2))
2018-12-21 14:24:25,225 - INFO - ('y_train', (1519, 6, 192, 2))


2018-12-21 14:28:27,632 - INFO - Val loss decrease from 1.5441 to 1.5434, saving to data/model/dcrnn_R_2_h_6_64-64_lr_0.01_bs_32_1221142424/models-1.5434-1488
2018-12-21 14:28:34,886 - INFO - Epoch [31/100] (1536) train_mae: 1.3500, val_mae: 1.5435 lr:0.000100 7.3s
2018-12-21 14:28:42,261 - INFO - Epoch [32/100] (1584) train_mae: 1.3500, val_mae: 1.5435 lr:0.000100 7.4s
2018-12-21 14:28:49,753 - INFO - Epoch [33/100] (1632) train_mae: 1.3503, val_mae: 1.5435 lr:0.000100 7.5s
2018-12-21 14:28:57,097 - INFO - Epoch [34/100] (1680) train_mae: 1.3500, val_mae: 1.5435 lr:0.000100 7.3s
2018-12-21 14:29:04,729 - INFO - Epoch [35/100] (1728) train_mae: 1.3500, val_mae: 1.5435 lr:0.000100 7.6s
2018-12-21 14:29:12,171 - INFO - Epoch [36/100] (1776) train_mae: 1.3500, val_mae: 1.5435 lr:0.000100 7.4s
2018-12-21 14:29:19,447 - INFO - Epoch [37/100] (1824) train_mae: 1.3499, val_mae: 1.5435 lr:0.000100 7.3s
2018-12-21 14:29:26,758 - INFO - Epoch [38/100] (1872) train_mae: 1.3499, val_mae: 1.5435 lr

2018-12-21 14:34:50,381 - INFO - Horizon 02, MAE: 1.514, MAPE: 2.4288, RMSE: 2.099
2018-12-21 14:34:50,386 - INFO - Horizon 03, MAE: 1.528, MAPE: 2.2135, RMSE: 2.119
2018-12-21 14:34:50,390 - INFO - Horizon 04, MAE: 1.536, MAPE: 2.1154, RMSE: 2.132
2018-12-21 14:34:50,394 - INFO - Horizon 05, MAE: 1.541, MAPE: 2.0030, RMSE: 2.139
2018-12-21 14:34:50,398 - INFO - Horizon 06, MAE: 1.546, MAPE: 2.1823, RMSE: 2.145
Predictions saved as ../DCRNN/movia/out.npz.


### diff 2 best
### dropuout 0.2, 0.5, 0 ingen effekt
###  1,3 layers no affect.
### 128 notes no affect
### laplace worse
### curric no affect
### learning rate no affect.
# BEST: diff2 trend removed, batch 32, no dual waak

### batch size 32 better, 16 worse/the same, 8 worse

### batch size 32
```
2018-12-10 19:18:20,415 - INFO - Horizon 01, MAE: 1.50, MAPE: 0.4415, RMSE: 2.08
2018-12-10 19:18:20,418 - INFO - Horizon 02, MAE: 1.57, MAPE: 0.4702, RMSE: 2.17
2018-12-10 19:18:20,421 - INFO - Horizon 03, MAE: 1.60, MAPE: 0.4847, RMSE: 2.22
2018-12-10 19:18:20,423 - INFO - Horizon 04, MAE: 1.63, MAPE: 0.5004, RMSE: 2.26
2018-12-10 19:18:20,426 - INFO - Horizon 05, MAE: 1.65, MAPE: 0.5073, RMSE: 2.30
2018-12-10 19:18:20,429 - INFO - Horizon 06, MAE: 1.66, MAPE: 0.5103, RMSE: 2.32
```
### batch size 16
```
2018-12-11 09:19:25,196 - INFO - Horizon 01, MAE: 1.48, MAPE: 0.4224, RMSE: 2.08
2018-12-11 09:19:25,200 - INFO - Horizon 02, MAE: 1.52, MAPE: 0.4571, RMSE: 2.15
2018-12-11 09:19:25,202 - INFO - Horizon 03, MAE: 1.54, MAPE: 0.4752, RMSE: 2.19
2018-12-11 09:19:25,205 - INFO - Horizon 04, MAE: 1.55, MAPE: 0.4830, RMSE: 2.21
2018-12-11 09:19:25,207 - INFO - Horizon 05, MAE: 1.56, MAPE: 0.4961, RMSE: 2.23
2018-12-11 09:19:25,210 - INFO - Horizon 06, MAE: 1.58, MAPE: 0.5134, RMSE: 2.25
```

### diff 2
```
2018-12-10 18:58:56,944 - WARNING - Early stopping at epoch: 80
2018-12-10 18:58:58,147 - INFO - Horizon 01, MAE: 1.48, MAPE: 0.4400, RMSE: 2.07
2018-12-10 18:58:58,150 - INFO - Horizon 02, MAE: 1.53, MAPE: 0.4668, RMSE: 2.14
2018-12-10 18:58:58,154 - INFO - Horizon 03, MAE: 1.55, MAPE: 0.4792, RMSE: 2.18
2018-12-10 18:58:58,157 - INFO - Horizon 04, MAE: 1.57, MAPE: 0.4917, RMSE: 2.21
2018-12-10 18:58:58,160 - INFO - Horizon 05, MAE: 1.59, MAPE: 0.5029, RMSE: 2.24
2018-12-10 18:58:58,163 - INFO - Horizon 06, MAE: 1.60, MAPE: 0.5120, RMSE: 2.27
```

### diff 2 no learning
```
2018-12-11 08:05:55,342 - INFO - Horizon 01, MAE: 1.48, MAPE: 0.4449, RMSE: 2.07
2018-12-11 08:05:55,345 - INFO - Horizon 02, MAE: 1.53, MAPE: 0.4698, RMSE: 2.14
2018-12-11 08:05:55,347 - INFO - Horizon 03, MAE: 1.55, MAPE: 0.4794, RMSE: 2.17
2018-12-11 08:05:55,350 - INFO - Horizon 04, MAE: 1.57, MAPE: 0.4924, RMSE: 2.21
2018-12-11 08:05:55,353 - INFO - Horizon 05, MAE: 1.58, MAPE: 0.5034, RMSE: 2.24
2018-12-11 08:05:55,355 - INFO - Horizon 06, MAE: 1.59, MAPE: 0.5096, RMSE: 2.26
```


### diff 2 - 3 layers
```
2018-12-10 20:41:26,454 - INFO - Horizon 01, MAE: 1.48, MAPE: 0.4389, RMSE: 2.07
2018-12-10 20:41:26,457 - INFO - Horizon 02, MAE: 1.52, MAPE: 0.4667, RMSE: 2.13
2018-12-10 20:41:26,460 - INFO - Horizon 03, MAE: 1.54, MAPE: 0.4764, RMSE: 2.17
2018-12-10 20:41:26,462 - INFO - Horizon 04, MAE: 1.56, MAPE: 0.4856, RMSE: 2.21
2018-12-10 20:41:26,465 - INFO - Horizon 05, MAE: 1.58, MAPE: 0.4960, RMSE: 2.23
2018-12-10 20:41:26,467 - INFO - Horizon 06, MAE: 1.59, MAPE: 0.5079, RMSE: 2.26
```

### diff 1  layer -- no affect
```
2018-12-10 20:58:01,709 - INFO - Horizon 01, MAE: 1.49, MAPE: 0.4421, RMSE: 2.07
2018-12-10 20:58:01,712 - INFO - Horizon 02, MAE: 1.54, MAPE: 0.4706, RMSE: 2.15
2018-12-10 20:58:01,715 - INFO - Horizon 03, MAE: 1.56, MAPE: 0.4851, RMSE: 2.18
2018-12-10 20:58:01,717 - INFO - Horizon 04, MAE: 1.58, MAPE: 0.5014, RMSE: 2.22
2018-12-10 20:58:01,720 - INFO - Horizon 05, MAE: 1.60, MAPE: 0.5108, RMSE: 2.25
2018-12-10 20:58:01,722 - INFO - Horizon 06, MAE: 1.61, MAPE: 0.5207, RMSE: 2.27
```

### diff 2 dropuout 0.2
```
2018-12-10 19:47:35,831 - INFO - Horizon 01, MAE: 1.48, MAPE: 0.4364, RMSE: 2.07
2018-12-10 19:47:35,834 - INFO - Horizon 02, MAE: 1.53, MAPE: 0.4638, RMSE: 2.14
2018-12-10 19:47:35,837 - INFO - Horizon 03, MAE: 1.54, MAPE: 0.4804, RMSE: 2.17
2018-12-10 19:47:35,839 - INFO - Horizon 04, MAE: 1.56, MAPE: 0.4963, RMSE: 2.21
2018-12-10 19:47:35,842 - INFO - Horizon 05, MAE: 1.58, MAPE: 0.5057, RMSE: 2.24
2018-12-10 19:47:35,845 - INFO - Horizon 06, MAE: 1.59, MAPE: 0.5155, RMSE: 2.26
```
### diff 2 dropuout 0.5
```
2018-12-10 19:47:29,754 - INFO - Horizon 01, MAE: 1.48, MAPE: 0.4453, RMSE: 2.07
2018-12-10 19:47:29,756 - INFO - Horizon 02, MAE: 1.53, MAPE: 0.4678, RMSE: 2.14
2018-12-10 19:47:29,759 - INFO - Horizon 03, MAE: 1.55, MAPE: 0.4828, RMSE: 2.17
2018-12-10 19:47:29,762 - INFO - Horizon 04, MAE: 1.57, MAPE: 0.4977, RMSE: 2.21
2018-12-10 19:47:29,764 - INFO - Horizon 05, MAE: 1.58, MAPE: 0.5072, RMSE: 2.24
2018-12-10 19:47:29,766 - INFO - Horizon 06, MAE: 1.60, MAPE: 0.5154, RMSE: 2.26
```
### diff 3
```
2018-12-10 19:01:31,102 - WARNING - Early stopping at epoch: 58
2018-12-10 19:01:32,378 - INFO - Horizon 01, MAE: 1.59, MAPE: 0.5304, RMSE: 2.21
2018-12-10 19:01:32,381 - INFO - Horizon 02, MAE: 1.60, MAPE: 0.5254, RMSE: 2.25
2018-12-10 19:01:32,384 - INFO - Horizon 03, MAE: 1.61, MAPE: 0.5227, RMSE: 2.27
2018-12-10 19:01:32,386 - INFO - Horizon 04, MAE: 1.62, MAPE: 0.5412, RMSE: 2.29
2018-12-10 19:01:32,389 - INFO - Horizon 05, MAE: 1.63, MAPE: 0.5460, RMSE: 2.32
2018-12-10 19:01:32,391 - INFO - Horizon 06, MAE: 1.63, MAPE: 0.5442, RMSE: 2.33
```


###  diff 2 128 notes
```
2018-12-10 21:18:15,171 - WARNING - Early stopping at epoch: 80
2018-12-10 21:18:16,450 - INFO - Horizon 01, MAE: 1.48, MAPE: 0.4415, RMSE: 2.07
2018-12-10 21:18:16,453 - INFO - Horizon 02, MAE: 1.53, MAPE: 0.4724, RMSE: 2.14
2018-12-10 21:18:16,456 - INFO - Horizon 03, MAE: 1.55, MAPE: 0.4848, RMSE: 2.18
2018-12-10 21:18:16,458 - INFO - Horizon 04, MAE: 1.56, MAPE: 0.4965, RMSE: 2.21
2018-12-10 21:18:16,461 - INFO - Horizon 05, MAE: 1.58, MAPE: 0.5045, RMSE: 2.24
2018-12-10 21:18:16,463 - INFO - Horizon 06, MAE: 1.59, MAPE: 0.5146, RMSE: 2.26
```


### diff2 laplacian 
```
2018-12-10 21:34:08,644 - INFO - Horizon 01, MAE: 1.50, MAPE: 0.4444, RMSE: 2.09
2018-12-10 21:34:08,647 - INFO - Horizon 02, MAE: 1.57, MAPE: 0.4723, RMSE: 2.18
2018-12-10 21:34:08,650 - INFO - Horizon 03, MAE: 1.60, MAPE: 0.4856, RMSE: 2.22
2018-12-10 21:34:08,652 - INFO - Horizon 04, MAE: 1.63, MAPE: 0.5040, RMSE: 2.26
2018-12-10 21:34:08,654 - INFO - Horizon 05, MAE: 1.65, MAPE: 0.5113, RMSE: 2.30
2018-12-10 21:34:08,657 - INFO - Horizon 06, MAE: 1.67, MAPE: 0.5134, RMSE: 2.33
```

### diff2 learning rate 0
```
2018-12-11 08:55:53,647 - INFO - Horizon 01, MAE: 1.48, MAPE: 0.4440, RMSE: 2.07
2018-12-11 08:55:53,650 - INFO - Horizon 02, MAE: 1.53, MAPE: 0.4749, RMSE: 2.15
2018-12-11 08:55:53,653 - INFO - Horizon 03, MAE: 1.55, MAPE: 0.4899, RMSE: 2.18
2018-12-11 08:55:53,656 - INFO - Horizon 04, MAE: 1.57, MAPE: 0.5047, RMSE: 2.22
2018-12-11 08:55:53,658 - INFO - Horizon 05, MAE: 1.59, MAPE: 0.5140, RMSE: 2.25
2018-12-11 08:55:53,661 - INFO - Horizon 06, MAE: 1.60, MAPE: 0.5242, RMSE: 2.28
```

### diff2 batch size 32
```
2018-12-11 09:10:56,256 - INFO - Horizon 01, MAE: 1.46, MAPE: 0.4336, RMSE: 2.05
2018-12-11 09:10:56,259 - INFO - Horizon 02, MAE: 1.50, MAPE: 0.4592, RMSE: 2.13
2018-12-11 09:10:56,261 - INFO - Horizon 03, MAE: 1.53, MAPE: 0.4805, RMSE: 2.16
2018-12-11 09:10:56,264 - INFO - Horizon 04, MAE: 1.54, MAPE: 0.4921, RMSE: 2.20
2018-12-11 09:10:56,266 - INFO - Horizon 05, MAE: 1.56, MAPE: 0.4977, RMSE: 2.22
2018-12-11 09:10:56,269 - INFO - Horizon 06, MAE: 1.57, MAPE: 0.5181, RMSE: 2.24
```

### diff2 trend removed, batch 32
```
2018-12-14 13:20:53,016 - INFO - Horizon 01, MAE: 1.49, MAPE: 3.7843, RMSE: 2.07
2018-12-14 13:20:53,019 - INFO - Horizon 02, MAE: 1.53, MAPE: 3.1711, RMSE: 2.13
2018-12-14 13:20:53,022 - INFO - Horizon 03, MAE: 1.55, MAPE: 2.8720, RMSE: 2.15
2018-12-14 13:20:53,024 - INFO - Horizon 04, MAE: 1.56, MAPE: 2.9472, RMSE: 2.16
2018-12-14 13:20:53,026 - INFO - Horizon 05, MAE: 1.56, MAPE: 3.0409, RMSE: 2.17
2018-12-14 13:20:53,029 - INFO - Horizon 06, MAE: 1.57, MAPE: 3.1860, RMSE: 2.18
```

### diff2 trend 
```
2018-12-14 14:12:45,195 - INFO - Horizon 01, MAE: 1.48, MAPE: 2.8957, RMSE: 2.05
2018-12-14 14:12:45,198 - INFO - Horizon 02, MAE: 1.51, MAPE: 2.5508, RMSE: 2.10
2018-12-14 14:12:45,201 - INFO - Horizon 03, MAE: 1.53, MAPE: 2.3908, RMSE: 2.13
2018-12-14 14:12:45,203 - INFO - Horizon 04, MAE: 1.54, MAPE: 2.2319, RMSE: 2.14
2018-12-14 14:12:45,206 - INFO - Horizon 05, MAE: 1.55, MAPE: 2.3854, RMSE: 2.15
2018-12-14 14:12:45,209 - INFO - Horizon 06, MAE: 1.55, MAPE: 2.4482, RMSE: 2.16
```

In [16]:
#new best [1.47019294 1.51492824 1.52761507 1.53462516 1.54025702 1.54603647] by adding identity matrix

In [17]:
### load data again
import numpy as np
with np.load('../DCRNN/movia/out.npz') as data:
    # also contains groundtruth
    print(np.abs(data['groundtruth'] - data['predictions']).mean(1).mean(1))

[1.4708338  1.51437517 1.5278665  1.53566043 1.54101168 1.54646748]


In [18]:
for i, r in enumerate(adj_mx):
    print(r)


[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0